In [19]:
pip install pyspark


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
#initiate spark instance 
import pyspark
from pyspark.sql.functions import col, sum
from pyspark.sql import SparkSession
#Create SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("SparkApp") \
    .getOrCreate()

#spark.stop()
print("spark success")

spark success


In [34]:
#print current directory
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current Working Directory: {current_directory}")


Current Working Directory: /Users/ruiiyoung/Downloads/MacAir2016_Desktop/IT stuff DIY +所有自己上的课/U St Andrews Data Science PGDip/5939data-driven system/assignment1/SparkA1


In [89]:
#part 1:1.load all csv in this same path
df= spark.read.format("csv").option("header", True).option("inferSchema", True).load("../lab_dataset.csv")
#df.show()
df.printSchema()


root
 |-- IncidentNumber: string (nullable = true)
 |-- DateOfCall: string (nullable = true)
 |-- CalYear: integer (nullable = true)
 |-- TimeOfCall: timestamp (nullable = true)
 |-- HourOfCall: integer (nullable = true)
 |-- IncidentGroup: string (nullable = true)
 |-- StopCodeDescription: string (nullable = true)
 |-- SpecialServiceType: string (nullable = true)
 |-- PropertyCategory: string (nullable = true)
 |-- PropertyType: string (nullable = true)
 |-- AddressQualifier: string (nullable = true)
 |-- Postcode_full: string (nullable = true)
 |-- Postcode_district: string (nullable = true)
 |-- UPRN: double (nullable = true)
 |-- USRN: integer (nullable = true)
 |-- IncGeo_BoroughCode: string (nullable = true)
 |-- IncGeo_BoroughName: string (nullable = true)
 |-- ProperCase: string (nullable = true)
 |-- IncGeo_WardCode: string (nullable = true)
 |-- IncGeo_WardName: string (nullable = true)
 |-- IncGeo_WardNameNew: string (nullable = true)
 |-- Easting_m: integer (nullable = true

24/08/08 08:08:46 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 402118 ms exceeds timeout 120000 ms
24/08/08 08:08:46 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/08 08:08:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$

In [87]:
#part1:2 Allow the user to search the dataset by the borough name, showing the total number of callouts in 2020.
# Filter the DataFrame to include only data for the year 2020
from pyspark.sql.functions import lit,upper, levenshtein #the functions used in this cell


df_2020 = df.filter(col('CalYear') == 2020)
# Group the data by 'IncGeo_BoroughName' and sum the 'NumbCalls' column
df_Borough_call = df_2020.groupBy('IncGeo_BoroughName', 'ProperCase').sum('NumCalls')\
                         .selectExpr( "`ProperCase` as `Borough Name CaseCorrect` ", "`sum(NumCalls)` as `Total_number_of_callouts`",)

#df_Borough_call.show(n=33, truncate=False)
# Function to search by borough name
def search_borough():
    userinput = input("Enter the borough name: ").strip()
    borough_input = userinput.upper()
    #give error if input is not letters
    if not borough_input.isalpha():
        print ("Borough name should be a string.")
        return
    try:
        result = df_Borough_call.filter(upper(col('Borough Name CaseCorrect')) == borough_input)
        # give error if the borough name is not in the dataset
        if result.count() == 0:
            # Check for the borough name is partial match
            partialmatch =df_Borough_call.filter(upper(col('Borough Name CaseCorrect')).contains(borough_input))
            if partialmatch.count() > 0:
                # Collect the matching borough names into a single list
                match_names = [row['Borough Name CaseCorrect'] for row in partialmatch.select('Borough Name CaseCorrect').collect()]
                # Print the matches in one line
                print(f"Partial match found. Did you mean: {', '.join(match_names)}")
                partialmatch.show(truncate=False)
            else:
                # Calculate Levenshtein distance for each borough name
                distances = df_Borough_call.withColumn('distance', levenshtein(upper(col('Borough Name CaseCorrect')), lit(borough_input)))
                # Find the closest match based on minimum distance
                closest_match = distances.select('Borough Name CaseCorrect', 'distance').orderBy('distance').first()
                closest_borough_name = closest_match['Borough Name CaseCorrect']
                print(f"No exact match found for '{userinput}'. Did you mean '{closest_borough_name}'? ")
                df_Borough_call.filter(col('Borough Name CaseCorrect') == closest_borough_name).show(truncate=False)
            return
        else:
            result.show(truncate=False)
    except Exception as e:
        print(f"Error: {e}")
# Call the function to take user input and display the result
search_borough()

Partial match found. Did you mean: City Of london
+------------------------+------------------------+
|Borough Name CaseCorrect|Total_number_of_callouts|
+------------------------+------------------------+
|City Of london          |1040                    |
+------------------------+------------------------+



In [86]:
# Wrong match demo for calculating distance

df_2020 = df.filter(col('CalYear') == 2020)
# Group the data by 'IncGeo_BoroughName' and sum the 'NumbCalls' column
df_Borough_call = df_2020.groupBy('IncGeo_BoroughName').sum('NumCalls')
df_Borough_call = df_Borough_call.withColumnRenamed('IncGeo_BoroughName', 'Borough Name')\
                                 .withColumnRenamed('sum(NumCalls)', 'Total number of callouts')

#df_Borough_call.show(n=33, truncate=False)
# Function to search by borough name
def search_borough():
    borough_name = input("Enter the borough name: ").strip().upper()
    #give error if input is not letters
    if not borough_name.isalpha():
        print ("Borough name should be a string.")
        return
    try:
        result = df_Borough_call.filter(col('Borough Name') == borough_name)
        # give error if the borough name is not in the dataset
        if result.count() == 0:
            # Calculate Levenshtein distance for each borough name
            distances = df_Borough_call.withColumn('distance', levenshtein(upper(col('Borough Name')), lit(borough_name)))
            #distances.show(truncate=False)
            # Find the closest match based on minimum distance
            closest_match = distances.select('Borough Name', 'distance').orderBy('distance').first()
            closest_borough_name = closest_match['Borough Name']

            
            print(f"No exact match found for '{borough_name}'. Did you mean '{closest_borough_name}'? ")
       
        else:
            result.show(truncate=False)
    
    except Exception as e:
        print(f"Error: {e}")
# Call the function to take user input and display the result
search_borough()

No exact match found for 'LONDON'. Did you mean 'CROYDON'? 


In [35]:
#part1: 3  Given a range of hours (for example between 1-2 hours), show the user all incidents where the LFB spent this much time dealing with it.
# Define the hours of interest as a list
hours_of_interest = [48,49]

# Filter DataFrame based on the HourOfCall and IncidentGroup conditions
df_time_spend = df.filter(
    (col("PumpHoursRoundUp").isin(hours_of_interest)) 
)
print(f"There are in total {df_time_spend.count()} incidents happened in 2021 which London Fire Brigade spent {hours_of_interest} hours dealing with it. Here are the detials:")
df_time_spend.show(truncate= False)

There are in total 15 incidents happened in 2021 which London Fire Brigade spent [48, 49] hours dealing with it. Here are the detials:


+---------------+-----------+-------+-------------------+----------+---------------+-------------------+---------------------------+----------------+-----------------------------------------------------+--------------------------------------------+-------------+-----------------+---------------+--------+------------------+--------------------+--------------------+---------------+------------------------------+------------------------------+---------+----------+---------------+----------------+-----------+------------+------+---------------------+--------------------------------+-------------------------------------+---------------------------------+--------------------------------------+-----------------------------+-----------------+---------+----------------+-----------------+--------+
|IncidentNumber |DateOfCall |CalYear|TimeOfCall         |HourOfCall|IncidentGroup  |StopCodeDescription|SpecialServiceType         |PropertyCategory|PropertyType                                        

In [36]:
#part1: 4 Calculate the number of incidents in 2021 where there were at least 2 fire pumps deployed.
# Filter DataFrame where PumpCount > 2
df_condition = df.filter((col("PumpCount") >=2 ) & (col("CalYear") == 2021)) 

# Count distinct PumpCount values
IncidentNumber = df_condition.select("IncidentNumber").distinct().count()
print(f"There are number of {IncidentNumber} incidents happened in 2021 which London Fire Brigade deployed at least 2 fire pumps")


There are number of 41572 incidents happened in 2021 which London Fire Brigade deployed at least 2 fire pumps


In [37]:
#part1:5.Which type of property had the most incident callouts during 2022?



# Filter the DataFrame to include only data for the year 2022
df_2022 = df.filter(col('CalYear') == 2022)
# Getting  the total number of incident calls for each property type
df_Property = df_2022.groupBy('PropertyType').agg(sum('NumCalls').alias('TotalNumCalls'))
df_Property = df_Property.orderBy(col('TotalNumCalls').desc())

df_Property_first =df_Property.first()
print(f"Property type: {df_Property_first['PropertyType']} had the most incident callouts during 2022")



Property type: Purpose Built Flats/Maisonettes - 4 to 9 storeys  had the most incident callouts during 2022


In [38]:
# Using an appropriate Python library, 
# draw a graph showing the trend of incident callouts on the road between 2020 and 2022. 
df_plot = df.filter(col("CalYear").isin([2020,2021,2022]))
# Make sure to define how you interpret a road incident based on the dataset.

#define road incident: road incident use PropertyCatergory as Road Vehicle, but there is also Road incident that happens without a vechile involved. 
#Therefoore PropertyType = Road surface/pavement, Roadside vegetation, Roadside furniture, Bridge. and PropertyCatergory = Road Vehicle
road_incidents = df_plot.filter(
    (col("PropertyCategory") == "Road Vehicle") | 
    (col("PropertyType").isin("Road surface/pavement", "Roadside vegetation", "Roadside furniture", "Bridge")) # these are in PropertyCatergory = "Outdoor"
)

#20629 with Road Vehicle Catergory, and with other condition, it is in 20988
import plotly.express as px
# draw a graph showing the trend of incident callouts on the road between 2020 and 2022

fig = px.line(road_incidents, x="DateOfCall", y="NumCalls", title="Trend of incident callouts on the road between 2020 and 2022")
# Update x-axis to show month marks only
fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Calls",
    xaxis=dict(
        tickformat="%b ",  # Format the ticks to show month and year
        dtick="M1"           # Show ticks every month
    )
)
fig.show()

In [3]:
# Using an appropriate Python library, 
# draw a graph showing the trend of incident callouts on the road between 2020 and 2022. 


# each block has a different assignment therefore the required pacakge is seperated 
from pyspark.sql.functions import  to_date, count,month, concat, lit 


df_plot = df.filter(col("CalYear").isin([2020,2021,2022]))
# Make sure to define how you interpret a road incident based on the dataset.

#define road incident: road incident use PropertyCatergory as Road Vehicle, but there is also Road incident that happens without a vechile involved. 
#Therefoore PropertyType = Road surface/pavement, Roadside vegetation, Roadside furniture, Bridge. and PropertyCatergory = Road Vehicle
road_incidents = df_plot.filter(
    (col("PropertyCategory") == "Road Vehicle") | 
    (col("PropertyType").isin("Road surface/pavement", "Roadside vegetation", "Roadside furniture", "Bridge")) # these are in PropertyCatergory = "Outdoor"
)

#20629 with Road Vehicle Catergory, and with other condition, it is in 20988

# Convert the 'DateOfCall' column from string to DateType
road_incidents = road_incidents.withColumn("DateOfCall", to_date(road_incidents["DateOfCall"], "dd MMM yyyy"))
#change the new added column also in the right type. Default is string: it will be 1, 10,11, 12, 2, 3, 4,
road_incidents = road_incidents.withColumn("MonthOfCall",  concat(month(road_incidents["DateOfCall"]).cast("integer")))
road_incidents = road_incidents.withColumn("YearMonthOfCall",concat(col("CalYear"), lit("-"), col("MonthOfCall")))

# # Group by year and month and count the number of records for each month
df_monthly_counts = road_incidents.groupBy("CalYear","MonthOfCall","YearMonthOfCall").agg(count("NumCalls").alias("NumCallsMonthly"))
df_monthly_counts = df_monthly_counts.withColumn("MonthOfCall", col("MonthOfCall").cast("integer"))
df_monthly_counts = df_monthly_counts.orderBy("CalYear","MonthOfCall")
#df_monthly_counts.show()

import plotly.express as px

fig = px.line(df_monthly_counts, x="YearMonthOfCall", y="NumCallsMonthly", title="Trend of LFR road incident callouts between 2020 and 2022")


fig.show()


In [40]:
pip install folium


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [43]:
#Part 2: gain a grade up to 16.5, 
import folium
from pyspark.sql.functions import col, count
#Where were the majority of incident callouts in the year 2020? Using suitable visualisa- tions, display the breakdown of incident callouts by location.
df_geo_plot = df.filter((col("CalYear") == 2020)& col("ProperCase").isNotNull())

# Aggregate data to get the number of calls per location
df_call_count = df_geo_plot.groupBy("ProperCase").agg(count("NumCalls").alias("NumCalls"))

#df_call_count.show()
# Collect the results for visualization
call_counts = df_call_count.collect()
print(call_counts[:5])

# Example dictionary of boroughs with their approximate lat-long positions
borough_locations = {
    # Add actual latitude and longitude coordinates for each borough
    'Westminster': (51.4975, -0.1357),
    'Southwark': (51.5036, -0.0803),
    'Camden': (51.545, -0.1628),
    'Croydon': (51.3762, -0.0982),
    'Lambeth': (51.4622, -0.1158),
    'Tower Hamlets': (51.5155, -0.0339),
    'Hackney': (51.545, -0.055),
    'Lewisham': (51.4452, -0.0209),
    'Ealing': (51.5132, -0.3089),
    'Barnet': (51.6251, -0.1517),
    'Kensington and Chelsea': (51.4942, -0.2018),
    'Haringey': (51.5908, -0.1119),
    'Islington': (51.5384, -0.1023),
    'Enfield': (51.6521, -0.0811),
    'Newham': (51.5306, 0.0398),
    'Hillingdon': (51.5333, -0.4663),
    'Brent': (51.5588, -0.2817),
    'Wandsworth': (51.4572, -0.191),
    'Greenwich': (51.4821, 0.0048),
    'Bromley': (51.4036, 0.0196),
    'Waltham Forest': (51.5908, -0.0118),
    'Hammersmith and Fulham': (51.4921, -0.2239),
    'Hounslow': (51.4661, -0.3529),
    'Redbridge': (51.5761, 0.0454),
    'Havering': (51.5742, 0.183),
    'Bexley': (51.4543, 0.1483),
    'Barking and Dagenham': (51.5452, 0.1557),
    'Merton': (51.4146, -0.191),
    'Harrow': (51.578, -0.3417),
    'Richmond upon Thames': (51.4613, -0.3037),
    'Sutton': (51.3618, -0.1948),
    'Kingston upon Thames': (51.4123, -0.3007),
    'City of London': (51.5155, -0.0922)
}

# Create a base map centered around London, UK
m = folium.Map(location=[51.509865, -0.118092], zoom_start=10)

# Add a marker for each borough with the number of calls
for row in call_counts:
    borough = row['ProperCase']
    calls = row['NumCalls']
    
    if borough in borough_locations:
        lat, lon = borough_locations[borough]
        folium.Marker(
            location=(lat, lon),
            popup=f"{borough}: {calls} calls",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

# Save the map to an HTML file and display it
m.save("borough_callouts_map.html")
m
# Postcode_full
# IncGeo_BoroughName
# ProperCase
# IncGeo_WardName
# Latitude
# Longitude


[Row(ProperCase='Wandsworth', NumCalls=2971), Row(ProperCase='Croydon', NumCalls=4148), Row(ProperCase='Bexley', NumCalls=2112), Row(ProperCase='City Of london', NumCalls=897), Row(ProperCase='Lambeth', NumCalls=4132)]


In [75]:
import folium
import json
from pyspark.sql.functions import col, count
from branca.colormap import linear

# Filter and aggregate data
df_geo_plot = df.filter((col("CalYear") == 2020) & col("ProperCase").isNotNull())
df_call_count = df_geo_plot.groupBy("ProperCase").agg(count("NumCalls").alias("NumCalls"))

# Collect results for visualization
call_counts = df_call_count.collect()

# Convert call_counts to a dictionary for easy lookup
call_count_dict = {row['ProperCase']: row['NumCalls'] for row in call_counts}
print(call_count_dict)

# Path to the GeoJSON file
geojson_file = '../london-boroughs_1179.geojson'

# Create a base map centered around London
m = folium.Map(location=[51.509865, -0.118092], zoom_start=10,tiles='OpenStreetMap')

# Normalize call counts for color mapping
min_calls = min(call_count_dict.values())
max_calls = max(call_count_dict.values())

# Create a color scale using linear colormap from branca

colormap = linear.YlOrRd_09.scale(min_calls, max_calls)

def style_function(feature):
    borough_name = feature['properties']['name']
    calls = call_count_dict.get(borough_name, 0)
    return {
        'fillColor': colormap(calls),
        'color': 'black',       # Outline color of polygons
        'weight': 2,           # Outline weight
        'fillOpacity': 0.6     # Fill opacity
    }



# Add GeoJSON layer to the map
folium.GeoJson(
    geojson_file,
    name='London Boroughs',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'], aliases=['Borough:'])
        

).add_to(m)





# Add color scale legend
colormap.caption = "Number of Calls"
colormap.add_to(m)

# Save the map to an HTML file and display
m.save("borough_polygons_map.html")
m


{'Wandsworth': 2971, 'Croydon': 4148, 'Bexley': 2112, 'City Of london': 897, 'Lambeth': 4132, 'Kensington And chelsea': 3240, 'Camden': 4154, 'Greenwich': 2882, 'Newham': 3143, 'Tower Hamlets': 4073, 'Kingston Upon thames': 1312, 'Hounslow': 2740, 'Barnet': 3344, 'Harrow': 1665, 'Islington': 3177, 'Brent': 3013, 'Haringey': 3227, 'Bromley': 2836, 'Merton': 1748, 'Westminster': 6812, 'Southwark': 4425, 'Hackney': 3937, 'Enfield': 3168, 'Ealing': 3374, 'Sutton': 1621, 'Richmond Upon thames': 1656, 'Havering': 2182, 'Hillingdon': 3142, 'Waltham Forest': 2820, 'Barking And dagenham': 1926, 'Redbridge': 2352, 'Lewisham': 3577, 'Hammersmith And fulham': 2760}


In [76]:
pip install dash dash-bootstrap-components plotly pyspark



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [82]:
import dash
from dash import html, dcc, callback, Output, Input
import dash_bootstrap_components as dbc
import plotly.express as px
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

# Create Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout of the app
app.layout = html.Div([
    # Dropdown for Borough selection
    dcc.Dropdown(
        id='borough-dropdown',
        options=[
            {'label': row['IncGeo_BoroughName'], 'value': row['IncGeo_BoroughName']}
            for row in df.select('IncGeo_BoroughName').distinct().collect()
        ],
        value=None,
        placeholder='Select a Borough'
    ),
    # Dropdown for Ward selection
    dcc.Dropdown(id='ward-dropdown', options=[], value=None, placeholder='Select a Ward'),
    
    # Dropdown for Postcode District selection
    dcc.Dropdown(id='postcode-district-dropdown', options=[], value=None, placeholder='Select a Postcode District'),
    
    # Dropdown for Postcode Full selection
    dcc.Dropdown(id='postcode-dropdown', options=[], value=None, placeholder='Select a Postcode'),

    # Graph for displaying data
    dcc.Graph(id='bar-chart')
])

# Callback for updating Ward dropdown based on Borough selection
@app.callback(
    Output('ward-dropdown', 'options'),
    Input('borough-dropdown', 'value')
)
def update_ward_dropdown(selected_borough):
    if selected_borough is None:
        return []
    
    # Filter DataFrame and collect ward options
    filtered_df = df.filter((col('IncGeo_BoroughName') == selected_borough) & (col('CalYear') == '2020'))
    wards = filtered_df.select('IncGeo_WardName').distinct().collect()
    options = [{'label': row['IncGeo_WardName'], 'value': row['IncGeo_WardName']} for row in wards]
    return options

# Callback for updating Postcode District dropdown based on Ward selection
@app.callback(
    Output('postcode-district-dropdown', 'options'),
    Input('ward-dropdown', 'value'),
    Input('borough-dropdown', 'value')
)
def update_postcode_district_dropdown(selected_ward, selected_borough):
    if selected_ward is None or selected_borough is None:
        return []
    
    # Filter DataFrame and collect postcode district options
    filtered_df = df.filter(
        (col('IncGeo_WardName') == selected_ward) &
        (col('IncGeo_BoroughName') == selected_borough) &
        (col('CalYear') == '2020')
    )
    postcode_districts = filtered_df.select('Postcode_district').distinct().collect()
    options = [{'label': row['Postcode_district'], 'value': row['Postcode_district']} for row in postcode_districts]
    return options
    

# Callback for updating Postcode dropdown based on Postcode District selection
@app.callback(
    Output('postcode-dropdown', 'options'),
    Input('postcode-district-dropdown', 'value'),
    Input('ward-dropdown', 'value'),
    Input('borough-dropdown', 'value')
)
def update_postcode_dropdown(selected_postcode_district, selected_ward, selected_borough):
    if selected_postcode_district is None or selected_ward is None or selected_borough is None:
        return []
    
    # Filter DataFrame and collect postcode options
    filtered_df = df.filter(
        (col('Postcode_district') == selected_postcode_district) &
        (col('IncGeo_WardName') == selected_ward) &
        (col('IncGeo_BoroughName') == selected_borough) &
        (col('CalYear') == '2020')
    )
    postcodes = filtered_df.select('Postcode_full').distinct().collect()
    # Correctly format options with both label and value
    options = [{'label': row['Postcode_full'], 'value': row['Postcode_full']} for row in postcodes]
    return options

# Callback for updating the bar chart based on dropdown selections
@app.callback(
    Output('bar-chart', 'figure'),
    Input('postcode-dropdown', 'value'),
    Input('postcode-district-dropdown', 'value'),
    Input('ward-dropdown', 'value'),
    Input('borough-dropdown', 'value')
)
def update_bar_chart(selected_postcode, selected_postcode_district, selected_ward, selected_borough):
    if selected_postcode:
        # Filter and aggregate data for the selected postcode
        filtered_df = df.filter(
            (col('Postcode_full') == selected_postcode) &
            (col('IncGeo_WardName') == selected_ward) &
            (col('IncGeo_BoroughName') == selected_borough) &
            (col('CalYear') == '2020')
        )
        postcode_counts = filtered_df.groupBy('Postcode_full').agg(sum('NumCalls').alias('Count')).collect()
        data = [{'Postcode_full': row['Postcode_full'], 'Count': row['Count']} for row in postcode_counts]
        fig = px.bar(data, x='Postcode_full', y='Count', title=f"NumCalls in {selected_postcode}")

    elif selected_postcode_district:
        # Filter and aggregate data for the selected postcode district
        filtered_df = df.filter(
            (col('Postcode_district') == selected_postcode_district) &
            (col('IncGeo_WardName') == selected_ward) &
            (col('IncGeo_BoroughName') == selected_borough) &
            (col('CalYear') == '2020')
        )
        district_counts = filtered_df.groupBy('Postcode_full').agg(sum('NumCalls').alias('Count')).collect()
        data = [{'Postcode_full': row['Postcode_full'], 'Count': row['Count']} for row in district_counts]
        fig = px.bar(data, x='Postcode_full', y='Count', title=f"NumCalls in {selected_postcode_district}")

    elif selected_ward:
        # Filter and aggregate data for the selected ward
        filtered_df = df.filter(
            (col('IncGeo_WardName') == selected_ward) &
            (col('IncGeo_BoroughName') == selected_borough) &
            (col('CalYear') == '2020')
        )
        ward_counts = filtered_df.groupBy('Postcode_district').agg(sum('NumCalls').alias('Count')).collect()
        data = [{'Postcode_district': row['Postcode_district'], 'Count': row['Count']} for row in ward_counts]
        fig = px.bar(data, x='Postcode_district', y='Count', title=f"NumCalls in {selected_ward}")

    elif selected_borough:
        # Filter and aggregate data for the selected borough
        filtered_df = df.filter(
            (col('IncGeo_BoroughName') == selected_borough) &
            (col('CalYear') == '2020')
        )
        borough_counts = filtered_df.groupBy('IncGeo_WardName').agg(sum('NumCalls').alias('Count')).collect()
        data = [{'IncGeo_WardName': row['IncGeo_WardName'], 'Count': row['Count']} for row in borough_counts]
        fig = px.bar(data, x='IncGeo_WardName', y='Count', title=f"NumCalls in {selected_borough}")

    else:
        # Aggregate data for all boroughs
        borough_counts = df.filter(col('CalYear') == '2020').groupBy('IncGeo_BoroughName').agg(sum('NumCalls').alias('Count')).collect()
        data = [{'IncGeo_BoroughName': row['IncGeo_BoroughName'], 'Count': row['Count']} for row in borough_counts]
        fig = px.bar(data, x='IncGeo_BoroughName', y='Count', title="NumCalls by Borough")

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [83]:
#Explore the callout times for these incidents and provide (with suitable visualisations) a summary of what you discover.
from pyspark.sql.functions import col,count

df_counts = df.groupBy("IncidentGroup", "HourofCall").agg(count("NumCalls").alias("NumCalls"))
# Sort by IncidentGroup and then by HourofCall
df_counts = df_counts.orderBy(col("IncidentGroup"), col("HourofCall"))
import plotly.express as px


# Create the line plot
fig = px.line(
    df_counts,
    x="HourofCall",
    y="NumCalls",
    color="IncidentGroup",
    title="Callout Times for Different Incident Groups",
    labels={"HourofCall": "Hour of Call", "NumCalls": "Number of Calls", "IncidentGroup": "Incident Group"}
)
fig.show()
